In [66]:
import bw2data as bd
import bw2calc as bc
import bw2io as bi
import bw2analyzer as ba

import numpy as np
import json, os
from pathlib import Path
import sys

In [2]:
!python --version

Python 3.11.0


In [3]:
bi.__version__

(0, 9, 'DEV14')

In [4]:
project_name = 'validation_ei39'
bd.projects.set_current(project_name)

In [5]:
bd.projects.current

'validation_ei39'

In [6]:
bd.databases

Databases dictionary with 2 object(s):
	biosphere3
	ecoinvent 3.9

In [7]:
# ecoinvent path and name
path_base = Path('/Users/ajakobs/Documents/data/')
fp_ecoinvent_39 = path_base / 'ecoinvent/ecoinvent 3.9_cutoff_ecoSpold02/datasets/'
ei39_name = "ecoinvent 3.9"

In [8]:
def import_ecoinvent(ei_path, ei_name):
    if ei_name in bd.databases:
        print(ei_name + " database already present!!! No import is needed")
    else:
        ei = bi.SingleOutputEcospold2Importer(ei_path, ei_name)
        ei.apply_strategies()
        ei.match_database(db_name='biosphere3', fields=('name', 'category', 'unit', 'location'))
        ei.statistics()
        ei.write_database()

In [9]:
# del bd.databases['biosphere3']
# del bd.databases[ei38_name]

In [10]:
bi.bw2setup()

Biosphere database already present!!! No setup is needed


In [11]:
bd.databases

Databases dictionary with 2 object(s):
	biosphere3
	ecoinvent 3.9

In [12]:
%%time
import_ecoinvent(fp_ecoinvent_39, ei39_name)

ecoinvent 3.9 database already present!!! No import is needed
CPU times: user 371 µs, sys: 377 µs, total: 748 µs
Wall time: 470 µs


In [8]:
ei = bd.Database(ei39_name)

# Checking Production volumes in ecoinvent

## Iron/Iron ore

In [115]:
# crude iron ore markets
crude_iron_markets = [(act, act['reference product']) for act in ei if 'iron ore, crude ore' in act['reference product'] and 'market' in act['name']]
crude_iron_markets

[('market for iron ore, crude ore, 46% Fe' (kilogram, GLO, None),
  'iron ore, crude ore, 46% Fe'),
 ('market for iron ore, crude ore, 63% Fe' (kilogram, GLO, None),
  'iron ore, crude ore, 63% Fe')]

In [10]:
# iron ore concentrate markets
iron_concentrate_market = [(act, act['reference product']) for act in ei if 'iron ore concentrate' in act['reference product'] and 'market' in act['name']]
iron_concentrate_market

[('market for iron ore concentrate' (kilogram, GLO, None),
  'iron ore concentrate')]

In [11]:
iron_concentrate_market[0][0].rp_exchange().as_dict()

{'flow': 'cb262c3c-a7c7-4781-82e6-e1e90dd92b65',
 'type': 'production',
 'name': 'iron ore concentrate',
 'classifications': {'CPC': ['14100: Iron ores and concentrates, other than roasted iron pyrites']},
 'production volume': 349547060065.289,
 'properties': {'carbon allocation': {'amount': 0.0, 'unit': 'kg'},
  'carbon content': {'amount': 0.0, 'unit': 'dimensionless'},
  'carbon content, fossil': {'amount': 0.0, 'unit': 'dimensionless'},
  'carbon content, non-fossil': {'amount': 0.0, 'unit': 'dimensionless'},
  'dry mass': {'amount': 1.0, 'unit': 'kg'},
  'price': {'amount': 0.0467, 'unit': 'EUR2005'},
  'water content': {'amount': 0.0, 'unit': 'dimensionless'},
  'water in wet mass': {'amount': 0.0, 'unit': 'kg'},
  'wet mass': {'amount': 1.0, 'unit': 'kg'}},
 'activity': '7bb8c2ac-d320-5d09-a311-8ce7f3e58fd7',
 'unit': 'kilogram',
 'amount': 1.0,
 'uncertainty type': 0,
 'loc': 1.0,
 'input': ('ecoinvent 3.9', '4c9e3b53b936ec11e1d2398488041820'),
 'output': ('ecoinvent 3.9', '4c

In [86]:
[act[0].rp_exchange().as_dict() for act in crude_iron_markets]


[{'flow': '686827fc-1f4c-44a4-a361-3ba77f0bc855',
  'type': 'production',
  'name': 'iron ore, crude ore, 46% Fe',
  'classifications': {'CPC': ['14100: Iron ores and concentrates, other than roasted iron pyrites']},
  'production volume': 631364276922.146,
  'properties': {'carbon allocation': {'amount': 0.0, 'unit': 'kg'},
   'carbon content': {'amount': 0.0, 'unit': 'dimensionless'},
   'carbon content, fossil': {'amount': 0.0, 'unit': 'dimensionless'},
   'carbon content, non-fossil': {'amount': 0.0, 'unit': 'dimensionless'},
   'dry mass': {'amount': 1.0, 'unit': 'kg'},
   'price': {'amount': 0.04, 'unit': 'EUR2005'},
   'water content': {'amount': 0.0, 'unit': 'dimensionless'},
   'water in wet mass': {'amount': 0.0, 'unit': 'kg'},
   'wet mass': {'amount': 1.0, 'unit': 'kg'}},
  'activity': '064a4f96-cd08-512b-99d7-70164cbe3a6f',
  'unit': 'kilogram',
  'amount': 1.0,
  'uncertainty type': 0,
  'loc': 1.0,
  'input': ('ecoinvent 3.9', 'c0f5b0e592eddf617e9e7e2601fba256'),
  'outp

In [16]:
prod_vols_iron_ore = [act[0].rp_exchange().as_dict()['production volume'] for act in crude_iron_markets]
prod_vols_iron_ore

[631364276922.146, 234920634920.635]

In [87]:
crude_iron_markets

[('market for iron ore, crude ore, 46% Fe' (kilogram, GLO, None),
  'iron ore, crude ore, 46% Fe'),
 ('market for iron ore, crude ore, 63% Fe' (kilogram, GLO, None),
  'iron ore, crude ore, 63% Fe')]

In [96]:
# now calculate the iron content
prod_vols_iron = [act[0].rp_exchange().as_dict()['production volume']*c for act,c in zip(crude_iron_markets,[0.46,0.63])]
prod_vols_iron

[290427567384.1872, 148000000000.00006]

In [99]:
# iron content of iron ore concentrate is 0.65 (see above), so add this to the iron prodction volume
prod_vols_iron.append(iron_concentrate_market[0][0].rp_exchange()['production volume']*0.65)
prod_vols_iron

[290427567384.1872, 148000000000.00006, 227205589042.43787]

In [100]:
# Total iron production volume (including iron from iron ore concentrates)
print(f'{sum(prod_vols_iron)/1e6} kt')

665633.1564266251 kt


In [26]:
# Total iron ore content (including iron ore concentrates)
print(f'{sum(prod_vols_iron_ore)/1e9} Mt')

1215.83197190807 Mt


## Copper:

In [37]:
copper_acts = [(act, act['reference product']) for act in ei if 'copper concentrate' in act['reference product']]
copper_acts


[('copper mine operation and beneficiation, sulfide ore' (kilogram, ZM, None),
  'copper concentrate, sulfide ore'),
 ('copper mine operation and beneficiation, sulfide ore' (kilogram, RU, None),
  'copper concentrate, sulfide ore'),
 ('copper mine operation and beneficiation, sulfide ore' (kilogram, ID, None),
  'copper concentrate, sulfide ore'),
 ('copper mine operation and beneficiation, sulfide ore' (kilogram, RoW, None),
  'copper concentrate, sulfide ore'),
 ('molybdenite mine operation' (kilogram, GLO, None),
  'copper concentrate, sulfide ore'),
 ('copper mine operation and beneficiation, sulfide ore' (kilogram, KZ, None),
  'copper concentrate, sulfide ore'),
 ('gold-silver mine operation and beneficiation' (kilogram, CA-QC, None),
  'copper concentrate, sulfide ore'),
 ('copper mine operation and beneficiation, sulfide ore' (kilogram, AU, None),
  'copper concentrate, sulfide ore'),
 ('market for copper concentrate, sulfide ore' (kilogram, GLO, None),
  'copper concentrate, 

In [44]:
copper_acts[-2][0].as_dict()

{'comment': 'The multi-output "zinc mine operation" process includes all steps required to produce zinc concentrate (59% Zn by mass), namely mining, comminution, and flotation. The dataset describes the production of zinc concentrate and additional co-products, primarily lead concentrate. Data is based on a study undertaken by the International Zinc Association (IZA) in conjunction with thinkstep (the LCA practitioner) for reference year 2012. Participating companies provided annual primary data on inputs and outputs for each process step, which was aggregated into a single production-weighted dataset. Zinc concentrate is most often used as an input to zinc smelting, either electrometallurgical smelting or the less common pyrometallurgical smelting. \nThe below image presents the system boundary in relation to the primary product, zinc concentrate.\nImage: https://db3.ecoinvent.org/images/2b83c8d3-c779-4e64-98f4-60d91c5dd741\nGeography:  This dataset is intended to be representative of

In [47]:
copper_acts[-2]

('zinc mine operation' (kilogram, GLO, None),
 'copper concentrate, sulfide ore')

In [49]:
copper_acts[-2][0].rp_exchange().as_dict()


{'flow': '0b4d69f9-6b39-4232-948b-62b7b4f0d3d8',
 'type': 'production',
 'name': 'copper concentrate, sulfide ore',
 'classifications': {'CPC': ['14290: Other non-ferrous metal ores and concentrates (other than uranium or thorium ores and concentrates)']},
 'production volume': 1769812941.63126,
 'properties': {'allocation factor': {'amount': 0.12493282773555899,
   'unit': 'dimensionless'},
  'aluminium content': {'amount': 0.00168, 'unit': 'dimensionless'},
  'barium content': {'amount': 8.92e-06, 'unit': 'dimensionless'},
  'carbon allocation': {'amount': 0.0, 'unit': 'kg'},
  'carbon content': {'amount': 0.0, 'unit': 'dimensionless'},
  'carbon content, fossil': {'amount': 0.0, 'unit': 'dimensionless'},
  'carbon content, non-fossil': {'amount': 0.0, 'unit': 'dimensionless'},
  'concentration, cadmium': {'amount': 5.3e-05, 'unit': 'kg/kg'},
  'concentration, calcium': {'amount': 0.0572, 'unit': 'kg/kg'},
  'concentration, copper': {'amount': 0.35, 'unit': 'kg/kg'},
  'concentration

In [59]:
copper_pVs = [(act[0]['name'],act[0].rp_exchange().as_dict()['production volume']*act[0].rp_exchange().as_dict()['properties']['copper content']['amount'],act[0].rp_exchange().as_dict()['properties']['copper content']['amount']) for act in copper_acts 
             if 'copper content' in act[0].rp_exchange().as_dict()['properties'].keys()]



In [60]:
copper_pVs

[('copper mine operation and beneficiation, sulfide ore',
  611374204.645576,
  0.296333333300647),
 ('copper mine operation and beneficiation, sulfide ore',
  811460242.4454681,
  0.296333333300648),
 ('copper mine operation and beneficiation, sulfide ore',
  637274275.4793061,
  0.296333333300647),
 ('copper mine operation and beneficiation, sulfide ore',
  5287710574.071248,
  0.296333333333333),
 ('molybdenite mine operation', 0.0, 0.0),
 ('copper mine operation and beneficiation, sulfide ore',
  517778274.26284724,
  0.296333333300648),
 ('gold-silver mine operation and beneficiation', 4504644.3816, 0.1188),
 ('copper mine operation and beneficiation, sulfide ore',
  1036001329.9257524,
  0.296333333300648),
 ('copper mine operation and beneficiation, sulfide ore',
  1856354281.0507677,
  0.296333333300648),
 ('primary zinc production from concentrate', 2253166.803325932, 0.65),
 ('copper mine operation and beneficiation, sulfide ore',
  883713547.0992817,
  0.296333333300647),
 (

In [39]:
sum(copper_pVs)/1e9

16.853553208441514

In [55]:
copper_market = [act for act in ei if act['name']=='market for copper concentrate, sulfide ore'][0]
copper_market

'market for copper concentrate, sulfide ore' (kilogram, GLO, None)

In [58]:
copper_market.rp_exchange().as_dict()

{'flow': '0b4d69f9-6b39-4232-948b-62b7b4f0d3d8',
 'type': 'production',
 'name': 'copper concentrate, sulfide ore',
 'classifications': {'CPC': ['14290: Other non-ferrous metal ores and concentrates (other than uranium or thorium ores and concentrates)']},
 'production volume': 67835027189.5556,
 'properties': {'carbon allocation': {'amount': 0.0, 'unit': 'kg'},
  'carbon content': {'amount': 0.0, 'unit': 'dimensionless'},
  'carbon content, fossil': {'amount': 0.0, 'unit': 'dimensionless'},
  'carbon content, non-fossil': {'amount': 0.0, 'unit': 'dimensionless'},
  'dry mass': {'amount': 1.0, 'unit': 'kg'},
  'price': {'amount': 0.8402003234447648, 'unit': 'EUR2005'},
  'water content': {'amount': 0.0, 'unit': 'dimensionless'},
  'water in wet mass': {'amount': 0.0, 'unit': 'kg'},
  'wet mass': {'amount': 1.0, 'unit': 'kg'}},
 'activity': 'a61f8c5b-9db2-53db-b695-c398fc1ec468',
 'unit': 'kilogram',
 'amount': 1.0,
 'uncertainty type': 0,
 'loc': 1.0,
 'input': ('ecoinvent 3.9', '1b76b

In [57]:
copper_market.rp_exchange()['production volume']*copper_market.rp_exchange()['properties']['copper content']['amount']

KeyError: 'copper content'

In [67]:
copper_smelting = [(act, act['reference product']) for act in ei if 'copper, anode' in act['reference product'] and 'smelting' in act['name']]
copper_smelting

[('smelting of copper concentrate, sulfide ore' (kilogram, CN, None),
  'copper, anode'),
 ('smelting of copper concentrate, sulfide ore' (kilogram, RoW, None),
  'copper, anode'),
 ('smelting of copper concentrate, sulfide ore' (kilogram, RU, None),
  'copper, anode'),
 ('smelting of copper concentrate, sulfide ore' (kilogram, IN, None),
  'copper, anode'),
 ('smelting of copper concentrate, sulfide ore' (kilogram, JP, None),
  'copper, anode'),
 ('smelting of copper concentrate, sulfide ore' (kilogram, CL, None),
  'copper, anode')]

In [69]:
print(sum([act[0].rp_exchange().as_dict()['production volume']*act[0].rp_exchange().as_dict()['properties']['copper content']['amount'] for act in copper_smelting 
             if 'copper content' in act[0].rp_exchange().as_dict()['properties'].keys()])/1e9, 'kt copper')

16.25580688573639 kt copper


In [70]:
[print(act[0], act[0].rp_exchange().as_dict()['production volume']*act[0].rp_exchange().as_dict()['properties']['copper content']['amount']/1e9, 'kt copper') for act in copper_smelting 
             if 'copper content' in act[0].rp_exchange().as_dict()['properties'].keys()]

'smelting of copper concentrate, sulfide ore' (kilogram, CN, None) 6.38422752154631 kt copper
'smelting of copper concentrate, sulfide ore' (kilogram, RoW, None) 6.256496034 kt copper
'smelting of copper concentrate, sulfide ore' (kilogram, RU, None) 0.7520371 kt copper
'smelting of copper concentrate, sulfide ore' (kilogram, IN, None) 0.4800555 kt copper
'smelting of copper concentrate, sulfide ore' (kilogram, JP, None) 1.1399616810340079 kt copper
'smelting of copper concentrate, sulfide ore' (kilogram, CL, None) 1.2430290491560714 kt copper


[None, None, None, None, None, None]

## Global copper production

24.823 kt of copper in 2021 according to: https://www.statista.com/statistics/254917/total-global-copper-production-since-2006/



## Aluminium

Aluminium is produced fron Bauxie, let's get the bauxite mining opertations

In [101]:
bauxite_mining = [(act, act['reference product']) for act in ei if 'bauxite' in act['reference product'] and 'mine operation' in act['name']]
bauxite_mining


[('bauxite mine operation' (kilogram, GLO, None), 'bauxite')]

In [102]:
bauxite_mining = bauxite_mining[0]

Let's have alook inside

In [64]:
bauxite_mining[0].as_dict()

{'comment': 'Average moisture content of bauxite is approximately 12% (wet mass basis).  The reference flow in this dataset is therefore expressed as 0.88 kg bauxite, without water.\n The aluminium content is based on the reported yield of aluminium oxide production (0.39 kg Al2O3/kg wet bauxite).  The aluminium hydroxide is also calculated from the subsequent aluminium oxide production yield and makes the simplifying assumption that all aluminium is bound in alumina trihydrates (Al(OH)3) since the split between alumina trihydrates and monohydrates is unknown.  The maximum error from this assumption is 23%.\nThe bauxite represents 50% (w/w) of the extracted ore, wet mass basis.\nGeography:  Data collected from 14 world-wide bauxite mines producing 143.1million metric tons of bauxite, representing about 50% of world-wide aluminium bauxite mining operations.\nTechnology:  Average bauxite mining technology world-wide (open mining)',
 'classifications': [('EcoSpold01Categories', 'metals/ex

In [103]:
# get the bauxite production volume
bauxite_mining[0].rp_exchange().as_dict()['production volume']/1e6

285.4831

And now the Aluminium PV in kt

In [168]:
bauxite_mining[0].rp_exchange().as_dict()['production volume']*bauxite_mining[0].rp_exchange().as_dict()['properties']['aluminium content']['amount']/1e6

68.97504187874488

This is a lot less then the 50% of the world wide production mentioned in the comments...

Now check the bauxite market to see if there's a higher production volume there..

In [79]:
bauxite_market = [act for act in ei if 'bauxite' in act['reference product'] and 'market for bauxite' in act['name']][0]


In [83]:
bauxite_market.rp_exchange().as_dict()['production volume']/1e6

285.4831

Nope... and by the looks of things below no al. content

In [84]:
bauxite_market.rp_exchange().as_dict()

{'flow': '6e6b1247-4f6e-44a8-b22f-7ff81a0e461a',
 'type': 'production',
 'name': 'bauxite',
 'classifications': {'CPC': ['14290: Other non-ferrous metal ores and concentrates (other than uranium or thorium ores and concentrates)']},
 'production volume': 285483100.0,
 'properties': {'carbon allocation': {'amount': 0.0, 'unit': 'kg'},
  'carbon content': {'amount': 0.0, 'unit': 'dimensionless'},
  'carbon content, fossil': {'amount': 0.0, 'unit': 'dimensionless'},
  'carbon content, non-fossil': {'amount': 0.0, 'unit': 'dimensionless'},
  'dry mass': {'amount': 1.0, 'unit': 'kg'},
  'price': {'amount': 0.033, 'unit': 'EUR2005'},
  'water content': {'amount': 0.0, 'unit': 'dimensionless'},
  'water in wet mass': {'amount': 0.0, 'unit': 'kg'},
  'wet mass': {'amount': 1.0, 'unit': 'kg'}},
 'activity': 'd061f63a-9447-5dbe-9cea-f4c36abc54f2',
 'unit': 'kilogram',
 'amount': 1.0,
 'uncertainty type': 0,
 'loc': 1.0,
 'input': ('ecoinvent 3.9', '9bb04916c547d1f289aac9b8faf49dc3'),
 'output': 

In [112]:
car = [act for act in ei if 'transport, passenger car, medium size, petrol, EURO 4' in act['name']][0]

In [113]:
car

'transport, passenger car, medium size, petrol, EURO 4' (kilometer, RoW, None)

In [114]:
[exc for exc in car.technosphere()]

[Exchange: -7.55254e-06 kilogram 'market for brake wear emissions, passenger car' (kilogram, GLO, None) to 'transport, passenger car, medium size, petrol, EURO 4' (kilometer, RoW, None)>,
 Exchange: 8.6021505376344e-06 unit 'market for passenger car maintenance' (unit, GLO, None) to 'transport, passenger car, medium size, petrol, EURO 4' (kilometer, RoW, None)>,
 Exchange: 0.0106666666666667 kilogram 'market for passenger car, petrol/natural gas' (kilogram, GLO, None) to 'transport, passenger car, medium size, petrol, EURO 4' (kilometer, RoW, None)>,
 Exchange: 0.0653567936 kilogram 'market for petrol, low-sulfur' (kilogram, RoW, None) to 'transport, passenger car, medium size, petrol, EURO 4' (kilometer, RoW, None)>,
 Exchange: 0.0009113964 meter-year 'market for road' (meter-year, GLO, None) to 'transport, passenger car, medium size, petrol, EURO 4' (kilometer, RoW, None)>,
 Exchange: 0.000265427119835466 meter-year 'market for road maintenance' (meter-year, RoW, None) to 'transport,